In [ ]:
from gomoku.learner import pexp_mind
from gomoku import core, models
from copy import deepcopy
from collections import defaultdict
import random
from core.board import Board

from pyspark.sql import Row
    
SIZE = 9
CHANNELS = 4

def versus(mind1_name, mind2_name, flipped):
    mind1 = pexp_mind.PExpMind(size=SIZE, init=False, channels=CHANNELS)
    mind1.load_net('../models/9_4_4')

    mind2 = pexp_mind.PExpMind(size=SIZE, init=False, channels=CHANNELS)
    mind2.load_net('../models/9_8_14_18')

    board = Board(size=SIZE, win_chain_length=5)
    
    # make a random move for first player
    for i in range(random.randint(0, int(SIZE * 2.5))):
        tourney_board.make_random_move()
    
    def expanding_p(depth, p):
        return np.logical_or(np.logical_or(
            np.logical_and(depth < 4, p > -5),
            np.logical_and(depth < 6, p > -4),
            np.logical_and(depth < 8, p > -4)),
            np.logical_and(depth < np.inf, p > -3)
        )

    def permissive_expansion(depth):
        if depth < 2:
            return np.inf
        if depth < 8:
            return 5
        return 3
        
    players = {}
    players[1] = mind1
    players[-1] = mind2
    board_strings = []

    while True:
        result = players[tourney_board.player_to_move].make_move(tourney_board, 
                                                                 as_player=tourney_board.player_to_move,
                                                                 retrain=False, 
                                                                 verbose=False, 
                                                                 epsilon=0.01, 
                                                                 max_depth=20,
                                                                 k=SIZE**2,
                                                                 max_iters=20)
        board_strings.append(board.pprint())
        
        if result:
            if tourney_board.game_won():
                wins[-tourney_board.player_to_move] += 1
            else:
                draws += 1
            break

    return Row(**{'boards' : board_strings, 
                  'wins_1': wins[1], 
                  'wins_2' : wins[-1], 
                  'draws' : draws, 
                  'flipped' : flipped})

SIZE=5


mind1 = pqmind.PQMind(size=SIZE, alpha=0.1, init=False)
mind1.load_net('gomoku/gomoku/models/distributed_5')
#mind2.load('models/9_9_q_v2_5.net', 'models/9_9_p_v2_5.net')

mind2 = pqmind.PQMind(size=SIZE, alpha=0.1, init=False)
mind2.load_net('gomoku/gomoku/models/distributed_5_bn')

mind1_name = '1_conv'
mind2_name = '3_conv_bn'

broadcast_net_bc = {}
broadcast_net_bc[mind1_name] = sc.broadcast((mind1.value_est.get_weights(), mind1.policy_est.get_weights()))
broadcast_net_bc[mind2_name] = sc.broadcast((mind2.value_est.get_weights(), mind2.policy_est.get_weights()))

rounds = 50

df = sc.parallelize([(mind1_name, mind2_name, True)] * rounds) \
        .union(sc.parallelize([(mind2_name, mind1_name, False)] * rounds)) \
        .map(lambda x : versus(x[0], x[1], x[2])).toDF()


    #print('Mind 1 Wins / Mind 2 Wins / Draws', wins[1], wins[-1], draws)

In [ ]:
sc.parallelize(zip(range(400), range(400))).partitionBy(400, lambda x: x).glom().collect()

In [ ]:
sc.parallelize([(mind1_name, mind2_name, True)] * rounds) \
        .union(sc.parallelize([(mind2_name, mind1_name, False)] * rounds)).take(10)

In [ ]:
from gomoku import models

In [ ]:
models.loader